Resposta 1 - Utilizando a biblioteca pandas.

In [2]:
import pandas as pd
import sqlite3
import tabula

In [7]:
#Extraindo cada tabela do pdf da avaliação tecnica com a biblioteca tabula.
dfs = tabula.read_pdf("tabelas.pdf", pages="all")

#Transformando cada tabela em um arquivo csv.
dfs[0].to_csv('pessoas.csv', index=False)
dfs[1].to_csv('contratos.csv', index=False)
dfs[2].to_csv('pagamentos.csv', index=False)

#Definindo cada dataframe apartir do csv gerado.
pessoas = pd.read_csv('pessoas.csv')
pagamentos = pd.read_csv('pagamentos.csv')
contratos = pd.read_csv('contratos.csv')

In [ ]:
#Mesclando as tabelas, e criando uma unica tabela.
df1 = pessoas.rename(columns={'ID': 'PESSOA_ID'})
df2 = pagamentos.rename(columns={'ID': 'PAGAMENTO_ID'})
df3 = contratos.rename(columns={'ID': 'CONTRATO_ID'})

df = df1.merge(df2).merge(df3)

In [ ]:
#Mudando o type da coluna 'DT_PAGAMENTO' e 'DT_COMPLETO' de object para datetime64
df['DT_PAGAMENTO'] = pd.to_datetime(df['DT_PAGAMENTO'])
df['DT_COMPLETO'] = pd.to_datetime(df['DT_COMPLETO'])
#Criando a coluna 'DIA_MES'
df['DIA_MES'] = df['DT_PAGAMENTO'].dt.day
#Criando a coluna 'VALOR_TOTAL'
df['VALOR_TOTAL'] = df.apply(lambda row: row['VALOR_PARCELA']*row['PARCELAS'] if pd.notna(row['DT_COMPLETO']) == True else 0, axis=1)


Selecionar o nome, dia do mês e valor da parcela para clientes que estão inadimplentes

In [ ]:
inadimplentes = df[df['INADIMPLENTE'] == 'S'][['NOME','DIA_MES','VALOR_PARCELA']].reset_index(drop=True)
inadimplentes

Selecionar o nome e valor total pago de clientes com pagamento completo.

In [ ]:
pg_completo = df[df['VALOR_TOTAL'] != 0][['NOME', 'VALOR_TOTAL']].reset_index(drop=True)
pg_completo

Resposta 2 - Utilizando SQL (Utilizamos o a biblioteca sqlite3 e pandas para visualizarmos o resultado direto no jupyter notebook)

Selecionar o nome, dia do mês e valor da parcela para clientes que estão inadimplentes

In [53]:
query = """WITH inadimplentes AS (
SELECT * FROM pessoas
JOIN contratos ON pessoas.CONTRATO_ID = contratos.id
JOIN pagamentos ON pessoas.ID = pagamentos.PESSOA_ID)
SELECT nome, STRFTIME('%d', dt_pagamento) as DIA_MES,valor_parcela from inadimplentes
WHERE inadimplente = 'S';
"""
dados = sqlite3.connect('dados.db') 
df = pd.read_sql(query, dados)
df


,NOME,DIA_MES,VALOR_PARCELA
0,Cristian Ghyprievy,19,300
1,Joana Cabel,25,150
2,John Serial,05,550


Selecionar o nome e valor total pago de clientes com pagamento completo.

In [54]:
query = """WITH pagamento_completo AS (
SELECT * FROM pessoas
JOIN contratos ON pessoas.CONTRATO_ID = contratos.id
JOIN pagamentos ON pessoas.ID = pagamentos.PESSOA_ID)
SELECT nome,(valor_parcela*parcelas) AS VALOR_TOTAL from pagamento_completo
WHERE dt_completo IS NOT NULL;
"""
dados = sqlite3.connect('dados.db') 
df = pd.read_sql(query, dados)
df

,NOME,VALOR_TOTAL
0,Michael Seven,14400
